# Logistic Regression
- **Task** - Classification: learn a function from one or more real value inputs to a categorical value for two (binary) or more (multi-class) categories/classes. 
- **Model/hypothesis** - a linear function

In [35]:
var _ = require('lodash');
var loader = require('csv-load-sync');
var d3 = require('d3');
var { JSDOM } = require('jsdom');
var {d3svg, d3bar, d3plot, d3fit, d3decision, argMax} = require('./utilities.js');

## Sample Data

In [36]:
var iris = loader('iris.csv')
iris[0]

{ Sepal_Length: '5.1',
  Sepal_Width: '3.5',
  Petal_Length: '1.4',
  Petal_Width: '0.2',
  Species: 'setosa' }

In [37]:
speciesColor = {setosa: '#dda0dd', versicolor: '#ffecdb', virginica: '#e08283'};
species = iris.map(o => { return {x: o.Petal_Length, y: o.Petal_Width, color: speciesColor[o.Species]}; });
d3plot(species, 'length', 'width');

## Binary classification
Is this flower of the "setosa" species?
- **Inputs (Features)** - petal length, petal width
- **Output (Label)** - 1 if setosa, 0 otherwise

In [38]:
function makeLabeled(data, species) {
    return iris.map(o => {
        return {
            x: [parseFloat(o.Petal_Length), parseFloat(o.Petal_Width)],
            y: (o.Species == species) ? 1 : 0
        };
    });
}
binarySpecies = makeLabeled(iris, 'setosa');
binarySpecies[0]

{ x: [ 1.4, 0.2 ], y: 1 }

## Sigmoid/Logit function

In [39]:
function sigmoid(t) {
    return 1/(1+Math.pow(Math.E, -t));
}
d3plot(_.range(-10,10,0.5).map(x => { return {x:x, y:sigmoid(x)}; }))

## Hypothesis
Same as before, but extended to two inputs and composed with sigmoid function.

- **Linear**: $f(x_0, x_1) = sigmoid(\alpha_0 x_0 + \alpha_1 x_1 + \beta)$
- **Parameters (Weights)**: p = $[\alpha_0, \alpha_0, \beta]$

In [40]:
var hypothesis = (x, p) => sigmoid(p[0] * x[0] + p[1] * x[1] + p[2]);

## Cost function

In [41]:
var calcCost = (data, model) => {
    var sum = 0;
    for (var pt of data) {
        var out = model(pt.x); 
        sum +=  -pt.y * Math.log(out) - (1 - pt.y) * Math.log(1 - out);
      }
      return sum / data.length;
}

## Training

In [42]:
function learnStep(data, model, initial, alpha) {
    var deltaSum = [0,0,0];

    var guess = initial;
    for (var pt of data) {
        var out = model(pt.x, guess);
        deltaSum[0] += (out - pt.y) * pt.x[0];
        deltaSum[1] += (out - pt.y) * pt.x[1];
        deltaSum[2] += out - pt.y;
    }

    var k = alpha / data.length;
    return [guess[0] - k * deltaSum[0], guess[1] - k * deltaSum[1], guess[2] - k * deltaSum[2]];
}
function learn(data, model, guess, alpha, steps) {
    for (var i = 0; i < steps; i++)
        guess = learnStep(data, hypothesis, guess, alpha);
    return guess;
}

In [43]:
var batch = 1000;
var guess = [0,0,0];
var learnCurve = [];
for (var i = 0; i < 15; i++) {
    var cost = calcCost(binarySpecies, x => hypothesis(x, guess));
    learnCurve.push(cost);
    console.log(guess, cost);
    guess = learn(binarySpecies, hypothesis, guess, 0.01, batch);
}
d3plot(learnCurve.map((c,i) => {return {x:i*batch,y:c};}), 'iterations', 'cost')

[ 0, 0, 0 ] 0.6931471805599463
[ -0.5489840554583205, -0.6177806017894323, 1.1203323545966797 ] 0.26296325322916836
[ -0.6959191671098105, -0.9566501811452067, 1.9491740194308198 ] 0.17982916021595935
[ -0.8164915167141275, -1.204357360389008, 2.5561375536006015 ] 0.13512517950896843
[ -0.9169698702716538, -1.3968192302188405, 3.028808337592586 ] 0.10796064785965769
[ -1.0025767456550563, -1.553315892748036, 3.4139311010680693 ] 0.08989688087623601
[ -1.0769795180657766, -1.6848647014505123, 3.7381939245183777 ] 0.07707269538257752
[ -1.1427192342881811, -1.7982126138146044, 4.017957050554574 ] 0.06751481956815628
[ -1.201591612832105, -1.897744988593167, 4.263872462734022 ] 0.06012164700329656
[ -1.2548995254927382, -1.9864550528281146, 4.483224576655352 ] 0.05423365145688828
[ -1.3036130143685871, -2.0664692119330406, 4.681198114459049 ] 0.04943325139009869
[ -1.348471462893898, -2.1393487427457663, 4.8616067201321425 ] 0.045443744934361444
[ -1.390050268230174, -2.2062719731118388, 

## Trained model

In [44]:
var predict = (x) => hypothesis(x, guess);
var correct = (o) => (predict([o.Petal_Length, o.Petal_Width]) > 0.5) == (o.Species=='setosa');
result = iris.map(o => { return {x:o.Petal_Length, y: o.Petal_Width, color: (correct(o)) ? 'green' : 'red'}; });
d3plot(result, 'length', 'width');

## Decision boundary

In [45]:
d3decision((x,y) => (predict([x,y]) > 0.5) ? 1 : 0, [1.0,7.5], [0,2.6], ['#ffecdb', '#dda0dd'], species)

## Multiclass classification

In [46]:
var setosaSpecies = makeLabeled(iris, 'setosa');
var versicolorSpecies = makeLabeled(iris, 'versicolor');
var virginicaSpecies = makeLabeled(iris, 'virginica');
var setosaGuess = learn(setosaSpecies, hypothesis, [0,0,0], 0.01, 25000);
var versicolorGuess = learn(versicolorSpecies, hypothesis, [0,0,0], 0.01, 25000);
var virginicaGuess = learn(virginicaSpecies, hypothesis, [0,0,0], 0.01, 25000);
[setosaGuess, versicolorGuess, virginicaGuess];

[ [ -1.7616770414592071, -2.784061217648509, 6.459255994065348 ],
  [ 1.1977096809817716, -2.3144776826230644, -2.4617661626246137 ],
  [ 0.15466956619449204, 4.178355103300627, -7.558526199031155 ] ]

In [47]:
var predict = (x) => [hypothesis(x, setosaGuess), hypothesis(x, versicolorGuess), hypothesis(x, virginicaGuess)];
predict([1.5, 0.3])

[ 0.951732339906592, 0.2043142704458409, 0.0022989175830190425 ]

In [48]:
var labels = ['setosa', 'versicolor', 'virginica'];
var correct = (o) => labels[argMax(predict([o.Petal_Length, o.Petal_Width]))] == o.Species;
result = iris.map(o => { return {x: o.Petal_Length, y: o.Petal_Width, color: (correct(o)) ? 'green' : 'red'}; });
d3plot(result, 'length', 'width');

In [49]:
d3decision((x,y) => argMax(predict([x,y])), [1.0,7.5], [0,2.6], ['#dda0dd', '#ffecdb', '#e08283'], species)